In [1]:
# ============================================================
# FAQ Dataset Preprocessing Pipeline
# Input:  dataset/single_qna.csv  (raw Amazon product Q&A)
# Output: dataset/faq_data.csv    (clean question + answer pairs)
# ============================================================

%pip install beautifulsoup4 --quiet

import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Load raw data
raw_df = pd.read_csv("dataset/single_qna.csv")
print(f"Raw dataset shape: {raw_df.shape}")
print(f"\nColumns: {raw_df.columns.tolist()}")
raw_df.head(3)

Note: you may need to restart the kernel to use updated packages.
Raw dataset shape: (1396896, 8)

Columns: ['QuestionType', 'Asin', 'AnswerTime', 'UnixTime', 'Question', 'AnswerType', 'Answer', 'Category']


,QuestionType,Asin,AnswerTime,UnixTime,Question,AnswerType,Answer,Category
0,yes/no,B00004U9JP,"Jun 27, 2014",1.403852e+09,I have a 9 year old Badger 1 that needs replac...,?,I replaced my old one with this without a hitch.,Appliances
1,open-ended,B00004U9JP,"Apr 28, 2014",1.398668e+09,model number,NaN,This may help InSinkErator Model BADGER-1: Bad...,Appliances
2,yes/no,B00004U9JP,"Aug 25, 2014",1.408950e+09,can I replace Badger 1 1/3 with a Badger 5 1/2...,?,Plumbing connections will vary with different ...,Appliances


In [2]:
# ============================================================
# STEP 1: Keep only relevant columns & initial inspection
# ============================================================

df = raw_df[['Question', 'Answer', 'Category']].copy()
print(f"Shape after selecting columns: {df.shape}")
print(f"\nNull values:\n{df.isnull().sum()}")
print(f"\nDuplicate rows: {df.duplicated().sum()}")
print(f"\nCategories: {df['Category'].nunique()}")
print(f"\nSample categories: {df['Category'].value_counts().head(10)}")

Shape after selecting columns: (1396896, 3)

Null values:
Question      1
Answer      146
Category      0
dtype: int64

Duplicate rows: 116675

Categories: 21

Sample categories: Category
Electronics                    314263
Home and Kitchen               184439
Sports and Outdoors            146891
Tools and Home Improvement     101088
Automotive                      89923
Cell Phones and Accessories     85865
Health and Personal Care        80496
Patio Lawn and Garden           59595
Toys and Games                  51486
Office Products                 43608
Name: count, dtype: int64


In [3]:
# ============================================================
# STEP 2: Drop nulls and duplicates
# ============================================================

df = df.dropna(subset=['Question', 'Answer'])
print(f"After dropping nulls: {df.shape}")

df = df.drop_duplicates(subset=['Question', 'Answer'])
print(f"After dropping duplicates: {df.shape}")

After dropping nulls: (1396749, 3)
After dropping duplicates: (1270873, 3)


In [4]:
# ============================================================
# STEP 3: Text cleaning function
# ============================================================

def clean_text(text):
    """Clean a single text string for FAQ use."""
    if not isinstance(text, str):
        return ""
    
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Replace multiple whitespace/newlines with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    return text

# Test the cleaner
sample = raw_df['Answer'].iloc[6]
print("BEFORE:", sample[:200])
print("\nAFTER:", clean_text(sample)[:200])

BEFORE: If your garbage disposal is leaking out the bottom near the reset button, you most likely have a leak in the metal plate that is located directly under the blades. This plate is designed to protect th

AFTER: If your garbage disposal is leaking out the bottom near the reset button, you most likely have a leak in the metal plate that is located directly under the blades. This plate is designed to protect th


In [5]:
# ============================================================
# STEP 4: Apply cleaning to Question and Answer columns
# ============================================================

df['Question'] = df['Question'].apply(clean_text)
df['Answer'] = df['Answer'].apply(clean_text)

print("Cleaning complete!")
df.head(5)

Cleaning complete!


,Question,Answer,Category
0,I have a 9 year old Badger 1 that needs replac...,I replaced my old one with this without a hitch.,Appliances
1,model number,This may help InSinkErator Model BADGER-1: Bad...,Appliances
2,can I replace Badger 1 1/3 with a Badger 5 1/2...,Plumbing connections will vary with different ...,Appliances
3,Does this come with power cord and dishwasher ...,It does not come with a power cord. It does co...,Appliances
4,loud noise inside when turned on. sounds like ...,Check if you dropped something inside.Usually ...,Appliances


In [6]:
# ============================================================
# STEP 5: Filter out junk / too-short answers
# ============================================================

# Remove rows where answer is just "?" or empty-ish
df = df[df['Answer'].str.strip() != '?']
df = df[df['Answer'].str.strip() != '']
df = df[df['Question'].str.strip() != '']

# Remove very short answers (< 10 chars) — likely useless
df = df[df['Answer'].str.len() >= 10]

# Remove very short questions (< 10 chars)
df = df[df['Question'].str.len() >= 10]

print(f"After filtering junk: {df.shape}")
print(f"\nSample questions:")
for q in df['Question'].head(5).tolist():
    print(f"  - {q[:100]}")

After filtering junk: (1173696, 3)

Sample questions:
  - I have a 9 year old Badger 1 that needs replacing, will this Badger 1 install just like the original
  - model number
  - can I replace Badger 1 1/3 with a Badger 5 1/2 - with same connections?
  - Does this come with power cord and dishwasher hook up?
  - loud noise inside when turned on. sounds like blades are loose


In [7]:
# ============================================================
# STEP 6: Remove duplicate questions (keep best answer — longest)
# ============================================================

# For FAQ, we want unique questions with the best answer
# Sort by answer length (descending) so the longest answer is kept
df['answer_len'] = df['Answer'].str.len()
df = df.sort_values('answer_len', ascending=False)

# Drop duplicate questions, keeping the first (longest answer)
df = df.drop_duplicates(subset=['Question'], keep='first')
df = df.drop(columns=['answer_len'])

print(f"After dedup by question: {df.shape}")

After dedup by question: (1113744, 3)


In [8]:
# ============================================================
# STEP 7: Truncate very long answers to avoid SBERT token limits
# ============================================================

MAX_ANSWER_LEN = 512  # characters — keeps it manageable for the model

df['Answer'] = df['Answer'].str[:MAX_ANSWER_LEN]

print(f"Answer length stats after truncation:")
print(df['Answer'].str.len().describe())

Answer length stats after truncation:
count    1.113744e+06
mean     1.574417e+02
std      1.322023e+02
min      1.000000e+01
25%      5.900000e+01
50%      1.160000e+02
75%      2.140000e+02
max      5.120000e+02
Name: Answer, dtype: float64


In [9]:
# ============================================================
# STEP 8: Sample to a manageable size (optional — for faster indexing)
# ============================================================
# The full dataset is ~1M+ rows. 
# For a production FAQ demo, 50K-100K is plenty.
# Adjust SAMPLE_SIZE as needed.

SAMPLE_SIZE = 50_000

if len(df) > SAMPLE_SIZE:
    df = df.sample(n=SAMPLE_SIZE, random_state=42)
    print(f"Sampled down to: {df.shape[0]} rows")
else:
    print(f"Dataset small enough ({df.shape[0]} rows), no sampling needed")

Sampled down to: 50000 rows


In [10]:
# ============================================================
# STEP 9: Rename columns to standard FAQ format & reset index
# ============================================================

faq_df = df[['Question', 'Answer']].copy()
faq_df.columns = ['question', 'answer']
faq_df = faq_df.reset_index(drop=True)

print(f"Final FAQ dataset shape: {faq_df.shape}")
print(f"Nulls: {faq_df.isnull().sum().sum()}")
faq_df.head(10)

Final FAQ dataset shape: (50000, 2)
Nulls: 0


,question,answer
0,"what size,typ or .....camera will fite to this...",Factory rear view camera
1,"How many inches wide do these tongs open, at t...",3 1/2 inches
2,Does it have alarm inputs and output?,Yes. you may configure several like loss of vi...
3,Is this the HPA300 model?,"No, this is the HPA100. It's smaller."
4,why is the burgundy FitBit $20 less than in bl...,I bought the burgundy FitBit because it was $2...
5,Can this unit be mounted on the wall as well? ...,aprox. 45 degrees
6,Does this kit including a cape?,"Yes it does come with a thin plastic cape, not..."
7,what is the actual gpm water usage?,"I don't know what the gpm is, I do know it is ..."
8,"Hi, What sizes do you have ? or is it an adjus...","Hi, it is the adjustable size. There is a tie ..."
9,"i need zinc with added selenium, does this cov...",Only ZINC Gluconate with cellulose Magnesium S...


In [11]:
# ============================================================
# STEP 10: Save the final FAQ dataset
# ============================================================

faq_df.to_csv("dataset/faq_data.csv", index=False)
print(f"Saved to dataset/faq_data.csv")
print(f"Total FAQ pairs: {len(faq_df)}")
print(f"\nFile size: {pd.io.common.file_exists('dataset/faq_data.csv')}")

Saved to dataset/faq_data.csv
Total FAQ pairs: 50000

File size: True


In [ ]:
# ============================================================
# STEP 11: Quick sanity check — verify the saved file
# ============================================================

check = pd.read_csv("dataset/faq_data.csv")
print(f"Loaded back: {check.shape}")
print(f"Columns: {check.columns.tolist()}")
print(f"\nRandom samples:")
for i, row in check.sample(5, random_state=1).iterrows():
    print(f"\n  Q: {row['question'][:80]}...")
    print(f"  A: {row['answer'][:80]}...")

Loaded back: (50000, 2)
Columns: ['question', 'answer']

Random samples:

  Q: T Mobile: Does this phone work with Tmobile?...
  A: Simple answer: yes. It does not come with a SIM card, however. If you are alread...

  Q: how is it on balls: does it nick...
  A: It works well. It shouldn't knick if you're careful. You just need to keep pulli...

  Q: Will this visor fit a VW CC 4 door ?...
  A: I'm not sure if the dimensions of your windshield are any different than our car...

  Q: how q;uickly will this unit inflate a low tire?...
  A: I would say about 5 minutes for a car tire that is low not flat...

  Q: I am unsure what is wrong with my set. I have plugged it in, and nothing is happ...
  A: The only other thing I would check are the volume knobs on the side of the earpi...


: 